In [1]:
import pandas as pd

### 2012 source file spec

In [2]:
# Define the file path
file_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\raw\kdkf_2012_raw.xls'

# Load the Excel file to get sheet names
xls = pd.ExcelFile(file_path)

# Define the years
previous_year = 2011
start_year = 2012
target_year = 2013

# Set the pandas option to display floating point numbers using the decimal format
pd.options.display.float_format = '{:.5f}'.format

### Master Loop

In [3]:
# Initialize an empty DataFrame for the master data
master_2012 = pd.DataFrame()

# Loop over each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Check if the sheet name starts with "HRM2_KT_"
    if sheet_name.endswith("HRM2"):
        # Load the sheet into a DataFrame, skipping the first three rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=0)

        # Merge the first row with the column headers
        df.columns = df.iloc[0, :2].tolist() + (df.columns[2:]).tolist()
        df = df.iloc[1:].reset_index(drop=True)

        # Update Headers of the df
        rename_dict = {
            'Rechnung': f'Rechnung {previous_year}',
            'Budget': f'Budget {start_year}',
            'Rechnung.1': f'Rechnung {start_year}',
            'Budget.1': f'Budget {target_year}'
        }
        df.rename(columns=rename_dict, inplace=True)

        # Rename the first column to 'HRM 2' and third column to "in 1000 CHF"
        df.columns.values[0] = 'HRM2-ID'
        df.columns.values[2] = 'HRM2-Bezeichnung'

        # Delete unecessary columns
        df.drop(df.columns[1], axis=1, inplace=True)
             
        # Add a new column with the DataFrame name (sheet name)
        df['Source'] = sheet_name
        df['Kanton'] = df['Source'].str.split(' HRM2').str[0]
        df['Year'] = 2012

        # Use pd.concat to add the DataFrame to the master DataFrame
        master_2012 = pd.concat([master_2012, df], ignore_index=True)

# Display the first few rows of the master DataFrame
master_2012.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2011,Budget 2012,Rechnung 2012,Budget 2013,Source,Kanton,Year,Unnamed: 7
0,ERFOLGSRECHNUNG,NaN,NaN,NaN,NaN,def.,ZH HRM2,ZH,2012,NaN
1,30,Personalaufwand,7369400.90000,4663146.94000,4782409.41000,4827007.19000,ZH HRM2,ZH,2012,NaN
2,31,Sach- und übriger Betriebsaufwand,2693614.80000,2513591.77000,2742025.90000,2633519.28000,ZH HRM2,ZH,2012,NaN
3,davon 314,baulicher und betrieblicher Unterhalt,192707,202022.00000,194592.14000,202593.45000,ZH HRM2,ZH,2012,NaN
4,davon 3180,Wertberichtigungen auf Forderungen,6103,2475.00000,-15285.00000,3557.70000,ZH HRM2,ZH,2012,NaN


### Translating FR columns to DE
Not needed, because there are no french cantons with HRM2 Reporting...

### Before saving the data, we drop non-numeric "HRM2-IDs", because they would cause issues with subsequent automated analysis...

In [4]:
# Drop the column named 'Unnamed: 7'
master_2012.drop('Unnamed: 7', axis=1, inplace=True)

# Drop rows where 'HRM-ID' values are non-numeric
master_2012 = master_2012[pd.to_numeric(master_2012['HRM2-ID'], errors='coerce').notna()]

# Display the first few rows of the updated DataFrame
len(master_2012)

1352

In [5]:
# Define the path where the CSV file will be saved
output_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\ready\kdkf_2012_data.csv'

# Save the DataFrame to a CSV file
master_2012.to_csv(output_path, index=False)
